In [1]:
import pandas as pd
import time
from datetime import datetime
import matplotlib.pyplot as plt
import talib as ta
from parinya import LINE

In [2]:
line = LINE('ccj5xXPxCpHhm1vcu6Z5bdnidwORoh10KpL9HkPYK91')

In [3]:
data = pd.read_csv(r'/Users/oat/Downloads/gold_clean.csv')

In [4]:
df = data.copy()

In [5]:
df.drop(columns='Unnamed: 0',inplace=True)
df.set_index('Date',inplace=True)
time = '2020'
df_0 = df.filter(regex=time,axis=0).filter(regex='O|H|L|C', axis=1)
df1 = df_0.filter(regex='-01-|-02-|-03-',axis=0).filter(regex='O|H|L|C', axis=1)
df2 = df_0.filter(regex='-04-|-05-|-06-',axis=0).filter(regex='O|H|L|C', axis=1)
df3 = df_0.filter(regex='-07-|-08-|-09-',axis=0).filter(regex='O|H|L|C', axis=1)
df4 = df_0.filter(regex='-10-|-11-|-12-',axis=0).filter(regex='O|H|L|C', axis=1)
df_0.reset_index(inplace = True)
df1.reset_index(inplace = True)
df2.reset_index(inplace = True)
df3.reset_index(inplace = True)
df4.reset_index(inplace = True)

In [6]:
df.set_index('Date',inplace=True)

In [7]:
time = '2020'
df_0 = df.filter(regex=time,axis=0).filter(regex='O|H|L|C', axis=1)
df1 = df_0.filter(regex='-01-|-02-|-03-',axis=0).filter(regex='O|H|L|C', axis=1)
df2 = df_0.filter(regex='-04-|-05-|-06-',axis=0).filter(regex='O|H|L|C', axis=1)
df3 = df_0.filter(regex='-07-|-08-|-09-',axis=0).filter(regex='O|H|L|C', axis=1)
df4 = df_0.filter(regex='-10-|-11-|-12-',axis=0).filter(regex='O|H|L|C', axis=1)
df_0.reset_index(inplace = True)
df1.reset_index(inplace = True)
df2.reset_index(inplace = True)
df3.reset_index(inplace = True)
df4.reset_index(inplace = True)

In [8]:
#input
x = df_0
close = x['Close']
high = x['High']
Open = x['Open']
low = x['Low']

# Model1

In [ ]:
def cross(df1,high,low,close):
    df = df1.copy()
    #import indicator
    df['STO_k'],df['STO_d'] = ta.STOCH(high,low,close,14,3,0,3,0)
    df['EMA_fast'] = ta.EMA(close,timeperiod = 12)
    df['EMA_slow'] = ta.EMA(close,timeperiod = 26)
    data = df.copy()
    value = 0
    lowest = 20
    highest = 80
    status = 'Close'
    sprade = 0.4
    #TP = 100
    #SL = 25
    for i in range(2,len(data)):
        I_past_1 = data.loc[i-2,'STO_k']
        I_past_2 = data.loc[i-2,'STO_d']
        I_1 = data.loc[i-1,'STO_k']
        I_2 = data.loc[i-1,'STO_d']

    

#ตัดลง 
        if I_past_1 > highest and I_1 < highest and status == 'Close' and data.loc[i-1,'EMA_fast'] > data.loc[i-1,'EMA_slow']:
            #data.loc[i,'STATUS'] = 'Open'
            data.loc[i,'status'] = 'Short'
            value = float(data.loc[i,'Open'])
            status = 'Open_Short'
        
        elif I_2 < I_1 and status == 'Open_Short':
            data.loc[i,'Value'] = value - (data.loc[i,'Open']+sprade)
            data.loc[i,'status'] = 'Close'
            status = 'Close'
               
#ตัดขึ้น        
        elif I_past_1 < lowest and I_1 > lowest and status == 'Close' and data.loc[i-1,'EMA_fast'] < data.loc[i-1,'EMA_slow']:
            data.loc[i,'status'] = 'Long'
            value = float(data.loc[i,'Open']) #commentละได้มากสุด
            status = 'Open_Long'
            
        elif I_1 < I_2 and status == 'Open_Long':
            data.loc[i,'status'] = 'Close'
            data.loc[i,'Value'] = data.loc[i,'Open'] - (value+sprade)
            status = 'Close' 
  
    
    for j in range(len(data)):
        if data.loc[j,'Value'] > 0:
            data.loc[j,'Win'] = 1
            data.loc[j,'Win_v'] = data.loc[j,'Value']
        elif data.loc[j,'Value'] < 0:
            data.loc[j,'Loss'] = 1
            data.loc[j,'Loss_v'] = data.loc[j,'Value']
        else:
            pass
            
    WIN_rate = data['Win_v'].sum()/abs(data['Loss_v'].sum())
    RR = (data['Win'].sum()/(data['Loss'].sum()+data['Win'].sum()))*100
    Win_value = data['Win_v'].sum()
    Loss_value = data['Loss_v'].sum()
    
    print(f'WIN rate: {WIN_rate}')   
    print(f'Risk reward: {RR}')  
    print(f'Win value : {Win_value}')
    print(f'Lose value : {Loss_value}')


# Model2

In [ ]:
def crossV2(df1,high,low,close,fast,slow,rsi,tp,sl): #TF day sideway
    df = df1.copy()
    #import indicator
    #df['STO_k'],df['STO_d'] = ta.STOCH(high,low,close,9,9,0,9,0)
    df['EMA_fast'] = ta.EMA(close,timeperiod = fast)
    df['EMA_slow'] = ta.EMA(close,timeperiod = slow)
    df['RSI'] = ta.RSI(close,timeperiod = rsi)
    #df['SAR'] = ta.SAR(high,low,0.05)
    data = df.copy()
    value = 0
    lowest = 30
    highest = 70
    status = 'Close'
    sprade = 0.4
    TP = tp
    SL = sl
    for i in range(2,len(data)):
#ตัดลง 
        if status == 'Close' and data.loc[i-1,'EMA_fast'] < data.loc[i-1,'EMA_slow'] and data.loc[i-2,'RSI'] > lowest and data.loc[i-1,'RSI'] < lowest:
            #data.loc[i,'STATUS'] = 'Open'
            data.loc[i,'status'] = 'Short'
            value = float(data.loc[i,'Open'])
            status = 'Open_Short'
        
        elif data.loc[i,'Open'] < value-TP and status == 'Open_Short':
            data.loc[i,'Value'] = value - (data.loc[i,'Open']+sprade)
            data.loc[i,'status'] = 'Close'
            status = 'Close'
        
        elif data.loc[i,'Open'] > value+SL and status == 'Open_Short':
            data.loc[i,'Value'] = value - (data.loc[i,'Open']+sprade)
            data.loc[i,'status'] = 'Close'
            status = 'Close'

#ตัดขึ้น        
        elif status == 'Close' and data.loc[i-1,'EMA_fast'] > data.loc[i-1,'EMA_slow'] and data.loc[i-2,'RSI'] < highest and data.loc[i-1,'RSI'] > highest:
            data.loc[i,'status'] = 'Long'
            value = float(data.loc[i,'Open']) #commentละได้มากสุด
            status = 'Open_Long'
            
        elif data.loc[i,'Open'] > value+TP and status == 'Open_Long':
            data.loc[i,'status'] = 'Close'
            data.loc[i,'Value'] = data.loc[i,'Open'] - (value+sprade)
            status = 'Close' 
        
        elif data.loc[i,'Open'] < value-SL and status == 'Open_Long':
            data.loc[i,'status'] = 'Close'
            data.loc[i,'Value'] = data.loc[i,'Open'] - (value+sprade)
            status = 'Close' 
  
    
    for j in range(len(data)):
        if data.loc[j,'Value'] > 0:
            data.loc[j,'Win'] = 1
            data.loc[j,'Win_v'] = data.loc[j,'Value']
        elif data.loc[j,'Value'] < 0:
            data.loc[j,'Loss'] = 1
            data.loc[j,'Loss_v'] = data.loc[j,'Value']
        else:
            pass
    
    P_L = data['Win_v'].sum()/abs(data['Loss_v'].sum())
    WIN_rate = (data['Win'].sum()/(data['Loss'].sum()+data['Win'].sum()))*100
    Win_value = data['Win_v'].sum()
    Loss_value = data['Loss_v'].sum()
    
    print(f'WIN rate: {WIN_rate}')   
    print(f'P/L: {P_L}')  
    print(f'Win value : {Win_value}')
    print(f'Lose value : {Loss_value}')
    

            
     

# Model3

In [ ]:
def crossV3(df1,high,low,close,ma_f,ma_s,cci): #TF day sideway
    df = df1.copy()
    #import indicator
    df['EMA_fast'] = ta.EMA(close,timeperiod = ma_f)
    df['EMA_slow'] = ta.EMA(close,timeperiod = ma_s)
    df['RSI'] = ta.RSI(close,timeperiod = 8)
    df['CCI'] = ta.CCI(high,low,close,cci)
    data = df.copy()
    value = 0
    lowest = 30
    highest = 70
    status = 'Close'
    sprade = 0.4
    #TP = 100
    #SL = 25
    #TP_status = False
    #SL_status = False
    for i in range(2,len(data)):
#ตัดลง 
        if status == 'Close' and data.loc[i-1,'EMA_fast'] < data.loc[i-1,'EMA_slow'] and data.loc[i-2,'EMA_fast'] > data.loc[i-2,'EMA_slow']:
            #data.loc[i,'STATUS'] = 'Open'
            data.loc[i,'status'] = 'Short'
            value = float(data.loc[i,'Open'])
            status = 'Open_Short'
        
            
        elif status == 'Open_Short' and data.loc[i-1,'CCI'] > 100:
            data.loc[i,'Value'] = value - (data.loc[i,'Open']+sprade)
            status = 'Close'

#ตัดขึ้น        
        elif status == 'Close' and data.loc[i-1,'EMA_fast'] > data.loc[i-1,'EMA_slow'] and data.loc[i-2,'EMA_fast'] < data.loc[i-2,'EMA_slow']:
            data.loc[i,'status'] = 'Long'
            value = float(data.loc[i,'Open']) #commentละได้มากสุด
            status = 'Open_Long'
            
        elif status == 'Open_Long' and data.loc[i-1,'CCI'] < -100:
            data.loc[i,'Value'] = value - (data.loc[i,'Open']+sprade)
            status = 'Close'
  
    
    for j in range(len(data)):
        if data.loc[j,'Value'] > 0:
            data.loc[j,'Win'] = 1
            data.loc[j,'Win_v'] = data.loc[j,'Value']
        elif data.loc[j,'Value'] < 0:
            data.loc[j,'Loss'] = 1
            data.loc[j,'Loss_v'] = data.loc[j,'Value']
        else:
            pass
    
    P_L = data['Win_v'].sum()/abs(data['Loss_v'].sum())
    WIN_rate = (data['Win'].sum()/(data['Loss'].sum()+data['Win'].sum()))*100
    Win_value = data['Win_v'].sum()
    Loss_value = data['Loss_v'].sum()
    
    print(f'WIN rate: {WIN_rate}')   
    print(f'P/L: {P_L}')  
    print(f'Win value : {Win_value}')
    print(f'Lose value : {Loss_value}')
    
    

# Model4

In [ ]:
def crossV4(df1,high,low,close,ma_f,ma_m,ma_s,rsi,tp): #TF dayเฟี้ยวปลายปี
    df = df1.copy()
    #import indicator
    df['EMA_fast'] = ta.EMA(close,timeperiod = ma_f)
    df['EMA_mid'] = ta.EMA(close,timeperiod = ma_m)
    df['EMA_slow'] = ta.EMA(close,timeperiod = ma_s)
    #df['STO_k'],df['STO_d'] = ta.STOCH(high,low,close,9,9,9)
    data = df.copy()
    value = 0
    lowest = 30
    highest = 70
    status = 'Close'
    sprade = 0.4
  
    
    for i in range(2,len(data)):
        
        TP = tp #10 point = 1000pip
        #SL = data.loc[i,'EMA_mid']
#short position
        if status == 'Close' and data.loc[i-1,'EMA_fast'] < data.loc[i-1,'EMA_mid'] < data.loc[i-1,'EMA_slow'] and abs(data.loc[i-1,'EMA_fast']-data.loc[i-1,'EMA_slow'])>1.5:
            if data.loc[i-2,'Close'] > data.loc[i-2,'EMA_fast'] and data.loc[i-1,'Close'] < data.loc[i-1,'EMA_fast']:
                #data.loc[i,'STATUS'] = 'Open'
                data.loc[i,'status'] = 'Short'
                value = float(data.loc[i,'Open'])
                status = 'Open_Short'
        
            
        elif data.loc[i,'Open'] < value-TP and status == 'Open_Short':
            data.loc[i,'Value'] = value - (data.loc[i,'Open']+sprade)
            data.loc[i,'status'] = 'Close'
            status = 'Close'
        
        elif data.loc[i,'Open'] > data.loc[i-1,'EMA_mid'] and status == 'Open_Short':
            data.loc[i,'Value'] = value - data.loc[i,'Open']
            data.loc[i,'status'] = 'Close'
            status = 'Close'

#long position        
        if status == 'Close' and data.loc[i-1,'EMA_fast'] > data.loc[i-1,'EMA_mid'] > data.loc[i-1,'EMA_slow'] and abs(data.loc[i-1,'EMA_fast']-data.loc[i-1,'EMA_slow'])>1.5:
            if data.loc[i-2,'Close'] < data.loc[i-2,'EMA_fast'] and data.loc[i-1,'Close'] > data.loc[i-1,'EMA_fast']:
                data.loc[i,'status'] = 'Long'
                value = float(data.loc[i,'Open']) #open แท่งต่่่อไป
                status = 'Open_Long'
            
        elif data.loc[i,'Open'] > value+TP and status == 'Open_Long':
            data.loc[i,'status'] = 'Close'
            data.loc[i,'Value'] = data.loc[i,'Open'] - (value+sprade)
            status = 'Close' 
        
        elif data.loc[i,'Open'] < data.loc[i-1,'EMA_mid'] and status == 'Open_Long':
            data.loc[i,'status'] = 'Close'
            data.loc[i,'Value'] = data.loc[i,'Open'] - value
            status = 'Close' 

    
    for j in range(len(data)):
        if data.loc[j,'Value'] > 0:
            data.loc[j,'Win'] = 1
            data.loc[j,'Win_v'] = data.loc[j,'Value']
        elif data.loc[j,'Value'] < 0:
            data.loc[j,'Loss'] = 1
            data.loc[j,'Loss_v'] = data.loc[j,'Value']
        else:
            pass
    
    P_L = data['Win_v'].sum()/abs(data['Loss_v'].sum())
    WIN_rate = (data['Win'].sum()/(data['Loss'].sum()+data['Win'].sum()))*100
    Win_value = data['Win_v'].sum()
    Loss_value = data['Loss_v'].sum()
    
    print(f'WIN rate: {WIN_rate}')   
    print(f'P/L: {P_L}')  
    print(f'Win value : {Win_value}')
    print(f'Lose value : {Loss_value}')
    
    
    return data

In [ ]:
s = crossV4(x,high,low,close,10,50,100,8,10)

In [ ]:
s = crossV4(x,high,low,close,10,50,100,8,5)

# Model 5

In [ ]:
#def Model5(): change sl
def crossV5(df1,high,low,close,ma_f,ma_m,ma_s,rsi,tp): 
    df = df1.copy()
    #import indicator
    df['EMA_fast'] = ta.EMA(close,timeperiod = ma_f)
    df['EMA_mid'] = ta.EMA(close,timeperiod = ma_m)
    df['EMA_slow'] = ta.EMA(close,timeperiod = ma_s)
    #df['STO_k'],df['STO_d'] = ta.STOCH(high,low,close,9,9,9)
    data = df.copy()
    value = 0
    lowest = 30
    highest = 70
    status = 'Close'
    sprade = 0.4
  
    
    for i in range(2,len(data)):
        
        TP = tp #10 point = 1000pip
        #SL = data.loc[i,'EMA_mid']
#short position
        if status == 'Close' and data.loc[i-1,'EMA_fast'] < data.loc[i-1,'EMA_mid'] < data.loc[i-1,'EMA_slow'] and abs(data.loc[i-1,'EMA_fast']-data.loc[i-1,'EMA_slow'])>1:
            if data.loc[i-2,'Close'] > data.loc[i-2,'EMA_fast'] and data.loc[i-1,'Close'] < data.loc[i-1,'EMA_fast']:
                #data.loc[i,'STATUS'] = 'Open'
                data.loc[i,'status'] = 'Short'
                value = float(data.loc[i,'Open'])
                status = 'Open_Short'
        
            
        elif data.loc[i,'Open'] < value-TP and status == 'Open_Short':
            data.loc[i,'Value'] = value - (data.loc[i,'Open']+sprade)
            data.loc[i,'status'] = 'Close'
            status = 'Close'
        
        elif data.loc[i-1,'EMA_fast'] > data.loc[i-1,'EMA_slow'] and status == 'Open_Short':
            data.loc[i,'Value'] = value - data.loc[i,'Open']
            data.loc[i,'status'] = 'Close'
            status = 'Close'

#long position        
        if status == 'Close' and data.loc[i-1,'EMA_fast'] > data.loc[i-1,'EMA_mid'] > data.loc[i-1,'EMA_slow'] and abs(data.loc[i-1,'EMA_fast']-data.loc[i-1,'EMA_slow'])>1:
            if data.loc[i-2,'Close'] < data.loc[i-2,'EMA_fast'] and data.loc[i-1,'Close'] > data.loc[i-1,'EMA_fast']:
                data.loc[i,'status'] = 'Long'
                value = float(data.loc[i,'Open']) #open แท่งต่่่อไป
                status = 'Open_Long'
            
        elif data.loc[i,'Open'] > value+TP and status == 'Open_Long':
            data.loc[i,'status'] = 'Close'
            data.loc[i,'Value'] = data.loc[i,'Open'] - (value+sprade)
            status = 'Close' 
        
        elif data.loc[i-1,'EMA_fast'] < data.loc[i-1,'EMA_slow'] and status == 'Open_Long':
            data.loc[i,'status'] = 'Close'
            data.loc[i,'Value'] = data.loc[i,'Open'] - value
            status = 'Close' 

    
    for j in range(len(data)):
        if data.loc[j,'Value'] > 0:
            data.loc[j,'Win'] = 1
            data.loc[j,'Win_v'] = data.loc[j,'Value']
        elif data.loc[j,'Value'] < 0:
            data.loc[j,'Loss'] = 1
            data.loc[j,'Loss_v'] = data.loc[j,'Value']
        else:
            pass
    
    P_L = data['Win_v'].sum()/abs(data['Loss_v'].sum())
    WIN_rate = (data['Win'].sum()/(data['Loss'].sum()+data['Win'].sum()))*100
    Win_value = data['Win_v'].sum()
    Loss_value = data['Loss_v'].sum()
    
    print(f'WIN rate: {WIN_rate}')   
    print(f'P/L: {P_L}')  
    print(f'Win value : {Win_value}')
    print(f'Lose value : {Loss_value}')
    
    
    return data

In [ ]:
s = crossV5(x,high,low,close,10,50,100,8,6)

In [ ]:
s = crossV5(x,high,low,close,10,30,50,8,6)

# Model 6 : Mid point

In [ ]:
def crossV6(df1,high,low,close,ma_f,ma_m,ma_s,rsi,tp): 
    df = df1.copy()
    #import indicator
    df['EMA_fast'] = ta.EMA(close,timeperiod = ma_f)
    df['EMA_mid'] = ta.EMA(close,timeperiod = ma_m)
    df['EMA_slow'] = ta.EMA(close,timeperiod = ma_s)
    #df['STO_k'],df['STO_d'] = ta.STOCH(high,low,close,9,9,9)
    data = df.copy()
    value = 0
    lowest = 30
    highest = 70
    status = 'Close'
    sprade = 0.4
  
    
    for i in range(2,len(data)):
        
        TP = tp #10 point = 1000pip
        #SL = data.loc[i,'EMA_mid']
#short position
        if status == 'Close' and data.loc[i-1,'EMA_fast'] < data.loc[i-1,'EMA_mid'] < data.loc[i-1,'EMA_slow'] and abs(data.loc[i-1,'EMA_fast']-data.loc[i-1,'EMA_slow'])>2:
            if data.loc[i-2,'Close'] > data.loc[i-2,'EMA_fast'] and data.loc[i-1,'Close'] < data.loc[i-1,'EMA_fast']:
                #data.loc[i,'STATUS'] = 'Open'
                data.loc[i,'status'] = 'Short'
                value = float(data.loc[i,'Open'])
                status = 'Open_Short'
        
            
        elif data.loc[i,'Open'] < value-TP and status == 'Open_Short':
            data.loc[i,'Value'] = TP-sprade
            data.loc[i,'status'] = 'Close'
            status = 'Close'
        
        elif data.loc[i-1,'Close'] > data.loc[i-1,'EMA_mid'] and status == 'Open_Short':
            data.loc[i,'Value'] = value - data.loc[i,'Open']
            data.loc[i,'status'] = 'Close'
            status = 'Close'

#long position        
        if status == 'Close' and data.loc[i-1,'EMA_fast'] > data.loc[i-1,'EMA_mid'] > data.loc[i-1,'EMA_slow'] and abs(data.loc[i-1,'EMA_fast']-data.loc[i-1,'EMA_slow'])>2:
            if data.loc[i-2,'Close'] < data.loc[i-2,'EMA_fast'] and data.loc[i-1,'Close'] > data.loc[i-1,'EMA_fast']:
                data.loc[i,'status'] = 'Long'
                value = float(data.loc[i,'Open']) #open แท่งต่่่อไป
                status = 'Open_Long'
            
        elif data.loc[i,'Open'] > value+TP and status == 'Open_Long':
            data.loc[i,'status'] = 'Close'
            data.loc[i,'Value'] = TP-sprade
            status = 'Close' 
        
        elif data.loc[i-1,'Close'] < data.loc[i-1,'EMA_mid'] and status == 'Open_Long':
            data.loc[i,'status'] = 'Close'
            data.loc[i,'Value'] = data.loc[i,'Open'] - value
            status = 'Close' 

    
    for j in range(len(data)):
        if data.loc[j,'Value'] > 0:
            data.loc[j,'Win'] = 1
            data.loc[j,'Win_v'] = data.loc[j,'Value']
        elif data.loc[j,'Value'] < 0:
            data.loc[j,'Loss'] = 1
            data.loc[j,'Loss_v'] = data.loc[j,'Value']
        else:
            pass
    
    P_L = data['Win_v'].sum()/abs(data['Loss_v'].sum())
    WIN_rate = (data['Win'].sum()/(data['Loss'].sum()+data['Win'].sum()))*100
    Win_value = data['Win_v'].sum()
    Loss_value = data['Loss_v'].sum()
    
    print(f'WIN rate: {WIN_rate}')   
    print(f'P/L: {P_L}')  
    print(f'Win value : {Win_value}')
    print(f'Lose value : {Loss_value}')
    line.sendtext(f'WIN rate: {WIN_rate}\nP/L: {P_L}')
    
    return data

In [ ]:
s = crossV6(x,high,low,close,10,30,50,8,6)

In [ ]:
s = crossV6(x,high,low,close,10,12,100,8,6)

In [9]:
def crossV7(df1,high,low,close,ma_f,ma_m,ma_s,stok,stod,stoks,tp,sl): 
    df = df1.copy()
    #import indicator
    df['EMA_fast'] = ta.EMA(close,timeperiod = ma_f)
    df['EMA_mid'] = ta.EMA(close,timeperiod = ma_m)
    df['EMA_slow'] = ta.EMA(close,timeperiod = ma_s)
    df['STO_k'],df['STO_d'] = ta.STOCH(high,low,close,14,3,3)
    #df['STO_k'],df['STO_d'] = ta.STOCH(high,low,close,9,9,9)
    data = df.copy()
    value = 0
    lowest = 30
    highest = 70
    status = 'Close'
    sprade = 0.4
    UP = 75
    Down = 25
  
    
    for i in range(2,len(data)):
        
        TP = tp #10 point = 1000pip
        SL = sl
#short position
        if status == 'Close' and data.loc[i-1,'EMA_fast'] < data.loc[i-1,'EMA_slow'] and abs(data.loc[i-1,'EMA_fast']-data.loc[i-1,'EMA_slow'])>0.5:
            if data.loc[i-2,'STO_k'] > data.loc[i-2,'STO_d'] and data.loc[i-1,'STO_k'] < data.loc[i-1,'STO_d'] and data.loc[i-1,'STO_k'] > UP:
                #data.loc[i,'STATUS'] = 'Open'
                data.loc[i,'status'] = 'Short'
                value = float(data.loc[i,'Open'])
                status = 'Open_Short'
        
            
        if data.loc[i,'Open'] < value-TP and status == 'Open_Short':
            data.loc[i,'Value'] = TP-sprade
            data.loc[i,'status'] = 'Close'
            status = 'Close'
        
        if data.loc[i,'Open'] > (value+sl) and status == 'Open_Short':
            data.loc[i,'Value'] = -SL
            data.loc[i,'status'] = 'Close'
            status = 'Close'

#long position        
        if status == 'Close' and data.loc[i-1,'EMA_fast'] > data.loc[i-1,'EMA_slow'] and abs(data.loc[i-1,'EMA_fast']-data.loc[i-1,'EMA_slow'])>0.5:
            if data.loc[i-2,'STO_k'] < data.loc[i-2,'STO_d'] and data.loc[i-1,'STO_k'] > data.loc[i-1,'STO_d'] and data.loc[i-1,'STO_k'] < Down:
                data.loc[i,'status'] = 'Long'
                value = float(data.loc[i,'Open']) #open แท่งต่่่อไป
                status = 'Open_Long'
            
        if data.loc[i,'Open'] > value+TP and status == 'Open_Long':
            data.loc[i,'status'] = 'Close'
            data.loc[i,'Value'] = TP-sprade
            status = 'Close' 
        
        if data.loc[i,'Open'] < (value-sl) and status == 'Open_Long':
            data.loc[i,'Value'] = -SL
            data.loc[i,'status'] = 'Close'
            status = 'Close'

    
    for j in range(len(data)):
        if data.loc[j,'Value'] > 0:
            data.loc[j,'Win'] = 1
            data.loc[j,'Win_v'] = data.loc[j,'Value']
        elif data.loc[j,'Value'] < 0:
            data.loc[j,'Loss'] = 1
            data.loc[j,'Loss_v'] = data.loc[j,'Value']
        else:
            pass
    
    P_L = data['Win_v'].sum()/abs(data['Loss_v'].sum())
    WIN_rate = (data['Win'].sum()/(data['Loss'].sum()+data['Win'].sum()))*100
    Win_value = data['Win_v'].sum()
    Loss_value = data['Loss_v'].sum()
    
    print(f'WIN rate: {WIN_rate}')   
    print(f'P/L: {P_L}')  
    print(f'Win value : {Win_value}')
    print(f'Lose value : {Loss_value}')
    line.sendtext(f'WIN rate: {WIN_rate}\nP/L: {P_L}')
    
    return data

In [10]:
s = crossV7(x,high,low,close,10,20,50,14,3,3,6,1)

WIN rate: 16.955445544554458
P/L: 1.1433681073025337
Win value : 2301.6000000000004
Lose value : -2013.0


In [11]:
s = crossV7(x,high,low,close,10,20,50,14,3,3,6,2)

WIN rate: 26.59423712801134
P/L: 1.0144144144144143
Win value : 3152.7999999999993
Lose value : -3108.0


In [12]:
line.sendtext('Finish')

'{"status":200,"message":"ok"}'

In [ ]:
s.to_excel('/Users/oat/Documents/Forex/test.xlsx')